# Calculate silicone phantom compositions

In [10]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline

# Calculate volume of Phantom from mould dimensions

In [11]:
# Units in mm
width = 50
depth = 50
thickness = 2

phantomVolReq = width * depth * thickness * 1E-4 # cm^3
print('Volume required: %.3f cm^3' % phantomVolReq)

Volume required: 0.500 cm^3


## Scatterer and Si weight
Calculate the required volume of silicone required to achieve the desired scattering coefficient calculated by Mie theory http://omlc.org/calc/mie_calc.html. 
#### Constants used in Mie theory calculation:

nMed = 1.41 (Silicone RTV)

nSphere = (material dependent)

lambda_0 = 1.54 um

sphereDiameter = x um

concentration in spheres / $\mu m^3$

#### Outputs: 
scattering coefficient ($\mu_s$) in $mm^{-1}$ and g factor.

In [12]:
# HELPER FUNCTIONS: 
def massDV(density, volume): return (density * volume)

def volSphere(diameter): return (4/3)*np.pi*(diameter/2)**3

# Calculate number of spheres in a gram given sphere diameter and mass
def numSpheres(mass, density, diameter): return  (mass/density) / (volSphere(diameter) * 1E-12)

# Convert concentrations given in online mie simulator (conc) and windows mie application (c):
def CtoConc(c): return c * 1E-9 # convert num sph in 1mm3 to spheres/um^3
def ConctoC(conc): return conc * 1E9 # convert spheres/um^3 to num sph in 1mm3

In [16]:
def SiliconeRequired(diameter, conc, sphDensity, sphPerGram, sphGram):
    '''
    diameter:     diameter of spheres in um
    conc:         spheres/micron^3 for required for desired scattering coefficient. Calculated from Mie theory.
    sphDensity :  density of the scattering material
    sphPerGram:   manufacurer stated number of spheres per gram in product
    sphGram:      number of grams to be used
    '''
    sphAdded = sphGram * sphPerGram    # Number of spheres added
    sphVol = volSphere(diameter)
    sphVolTotal = sphAdded * sphVol    # Total volume of scatterers added
    
    sphConc = sphVol*conc              # Required fraction of scatterers
    siConc = 1 - sphConc               # Required fraction of silicone
    print('Concentration of silicone in sample: %.2f' % (100 * siConc))
    
    siVol = sphVolTotal * (1/sphConc - 1) # Required volume of silicone ( from C1 = V1/(V1+V2) )
    siDensity = 0.97                      # g/cm^3, ELASTOSIL® RT 601 A/B
    
    siliconeMass = massDV((siDensity), siVol*1E-12)   # Density in same units as siVol, i.e. as diameter
    sphMass = massDV((sphDensity), sphVolTotal*1E-12) 
    print('Volume of a sphere: %.3f um^3' % sphVol)
#     print('Volume of 1g of spheres: %.3g um^3' % sphVolTotal)
#     print('Volume of silicone required: %.3g' % siVol)
    weightPercent = 100* sphMass / (siliconeMass + sphMass)
    print('Weight percent of scatters: %.f %%' % weightPercent)
    
    assert abs(siVol / (sphVolTotal + siVol) - siConc) < 0.001, 'Check concentrations'
    
    print('\n\nSilicone required: %.2g g' % (siliconeMass))
    phantomVol = (sphVolTotal + siVol) * 1E-12
    print('Final volume of phantom: %.3f cm^3' % phantomVol)

    numMoulds = phantomVol / phantomVolReq
    print('Number of moulds can make: %.2f' % numMoulds)
    

### SiO2 Scatterers
nSphere = 1.4441 at 1.54 um (SiO2 http://refractiveindex.info/?shelf=main&book=SiO2&page=Malitson)
g = 0.89

For a scattering coefficient of 12 $mm^{-1}$ (equivalent to the dermis and epidermis), require 0.4 spheres per cubic micron ($um^3$)

In [17]:
diameter = 2 # um
conc = 0.1  # Required number of spheres per um^3
sphDensity = 2.65  # g/cm^3
sphPerGram = 3E11  # Number of spheres in 1g (£100)
sphGram = 1  # weight of scatterers added in g

SiliconeRequired(diameter, conc, sphDensity, sphPerGram, sphGram)

Concentration of silicone in sample: 58.11
Volume of a sphere: 4.189 um^3
Weight percent of scatters: 66 %


Silicone required: 1.7 g
Final volume of phantom: 3.000 cm^3
Number of moulds can make: 6.00


### Polystyrene - Polybead
nSphere = 1.57 : http://www.polysciences.com/german/polybeadsupsup-microspheres-dry-form-150956m

In [32]:
diameter = 1.5  # in um
sphDensity = 1.05  # Polystyrene g/cm^3
conc = 0.02  # Required number of spheres per um^3
sphGram = 0.5  # weight of scatterers added in g (£165)
sphPerGram = numSpheres(sphGram, sphDensity, diameter)  # Number of spheres in 1g

print('Number of spheres in sample: %.3g' % (sphPerGram*sphGram))

SiliconeRequired(diameter, conc, sphDensity, sphPerGram, sphGram)

Number of spheres in sample: 1.35e+11
Concentration of silicone in sample: 96.47
Volume of a sphere: 1.767 um^3
Weight percent of scatters: 4 %


Silicone required: 6.3 g
Final volume of phantom: 6.737 cm^3
Number of moulds can make: 13.47


In [20]:
# Using mie simulator windows concistency check
c = 1.5E7                    # num sph in 1mm3 phantom in standalone form#

totsph = 2.7E11/2            # Total number of spheres
tvol = c * volSphere(1.5E-3) # sph vol/1mm3 phantom
siv = 1 - tvol               # silicone vol/1mm^3 phantom

mm3 = (totsph/c)             # total mm3 can make with 0.5g (2.7E11 spheres)

stot = mm3 * siv             # mm3 of silicone needed (total phantom volume in mm * fraction of silicone)
cm3 = stot * 1E-3            # total cm3

smasss = 0.97 * cm3          # m = rho*v (si rho in g/cm^3)
print('%.3g g of silicone' % smasss)    # g of silicone

CtoConc(c)

8.5 g of silicone


0.015000000000000001

### TiO2
nSphere = 2.45

In [34]:
diam = 0.25 # um
conc = 0.2  # Required number of spheres per um^3
sphDensity = 4.23
sphGram = 0.5  # weight of scatterers added in g
sphPerGram = numSpheres(sphGram, sphDensity, diameter)  # Number of spheres in 1g

SiliconeRequired(diam, conc, sphDensity, sphPerGram, sphGram)

Concentration of silicone in sample: 99.84
Volume of a sphere: 0.008 um^3
Weight percent of scatters: 1 %


Silicone required: 0.16 g
Final volume of phantom: 0.167 cm^3
Number of moulds can make: 0.33


### Polyethylene
From Cospheric 200mg, 1 - 4 um diameter for $200. n = 1.48.

In [30]:
diameter = 2.5  # in um
sphDensity = 0.96  # Polystyrene g/cm^3
conc = 0.01  # Required number of spheres per um^3
sphGram = 0.2  # weight of scatterers added in g
sphPerGram = numSpheres(sphGram, sphDensity, diameter)  # Number of spheres in 1g

print('Number of spheres in sample: %.3g' % (sphPerGram*sphGram))

SiliconeRequired(diameter, conc, sphDensity, sphPerGram, sphGram)

Number of spheres in sample: 5.09e+09
Concentration of silicone in sample: 91.82
Volume of a sphere: 8.181 um^3
Weight percent of scatters: 8 %


Silicone required: 0.45 g
Final volume of phantom: 0.509 cm^3
Number of moulds can make: 1.02
